<a href="https://colab.research.google.com/github/hewp84/CRT420/blob/main/Pandas_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PANDAS: Grouping & Reshaping

## Grouping

Grouping is a fundamental operation that allows us to split our data into groups based on specific criteria, and then perform operations on each group separately. 

**Syntax:**

`df.groupby(by=None, axis=0, level=None, as_index=True, sort=True, group_keys=True, squeeze=<no_default>, observed=False, dropna=True)`

**Parameters:**

* by : Mapping, function, str, or iterable to group by. Column name/s or index level/s to group.
* axis : Axis to group over. 0 for rows and 1 for columns. Default is 0.
* level : Level of MultiIndex to group.
* as_index : Group by index if True, else by values. Default is True.
* sort : Sort groups if True. Default is True.
* group_keys : Add group keys to index. Default is True.
* squeeze : Return NumPy value if possible.
* observed : Use only observed values.
* dropna : Don't include NaN values. Default is True.

**Returns:**

A groupby object that contains information about the groups.

In [95]:
# Import necessary libraries
import pandas as pd

# Create a sample DataFrame
data = {
    'Category': ['A', 'B', 'A', 'B', 'A', 'B','C'],
    'Value': [10, 15, 20, 25, 30, 35, 67]
}

df = pd.DataFrame(data)

# Display the DataFrame
df


,Category,Value
0,A,10
1,B,15
2,A,20
3,B,25
4,A,30
5,B,35
6,C,67


To group data in Pandas, we can use the `groupby()` method, which is often followed by an aggregation operation. Let's start by grouping our sample DataFrame `df` by the 'Category' column.


In [96]:
# Group the DataFrame by 'Category'
grouped = df.groupby(by='Category')

# Display the grouped object
grouped

In [83]:
# Calculate the mean value for each group
grouped_mean = grouped.mean()

# Display the mean values
grouped_mean


,Value
Category,
A,20.0
B,25.0


In [116]:
#Try alternative mathematical methods such as sum, mean, etc
grouped_example = grouped.max()

grouped_example

,Value
Category,
A,30
B,35
C,67


#### Aggregation

We can also iterate through the groups and perform custom operations on each group. Let's print the groups and their corresponding data:

In [87]:
# Iterate through groups and print the group name and data
for name, group in grouped:
    print(f"Group: {name}")
    print(group)
    print()


Group: A
  Category  Value
0        A     10
2        A     20
4        A     30

Group: B
  Category  Value
1        B     15
3        B     25
5        B     35



#### Applying Multiple Aggregations

Pandas allows us to apply multiple aggregation functions at once using the `agg()` method. Let's calculate both the mean and sum for each group.

In [105]:

# Calculate both mean and sum for each group
grouped_agg = grouped['Value'].agg(['mean', 'sum','max'])

# Display the aggregated results
grouped_agg


,mean,sum,max
Category,,,
A,20.0,60,30
B,25.0,75,35
C,67.0,67,67


#### Custom Aggregation Functions

You can define custom aggregation functions to apply to groups. Let's create a custom function to calculate the range for each group.

In [106]:
# Custom aggregation function to calculate the range
def custom_range(x):
    return x.max()-x.min()

# Apply the custom aggregation function
range_result = grouped['Value'].agg(custom_range)

# Display the range for each group
range_result


Category
A    20
B    20
C     0
Name: Value, dtype: int64

#### Real life dataset: ATP Tour 2013-2023

In [107]:
atp = pd.read_csv('atp_tennis.csv')
atp

,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,score
0,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Mayer F.,Giraldo S.,Mayer F.,28,57,1215,778,1.36,3.00,6-4 6-4
1,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Benneteau J.,Nieminen J.,Nieminen J.,35,41,1075,927,2.20,1.61,3-6 6-2 1-6
2,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Nishikori K.,Matosevic M.,Nishikori K.,19,49,1830,845,1.25,3.75,7-5 6-2
3,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Mitchell B.,Baghdatis M.,Baghdatis M.,326,36,137,1070,9.00,1.07,4-6 4-6
4,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3,Istomin D.,Klizan M.,Istomin D.,43,30,897,1175,1.90,1.80,6-1 6-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25357,Miami Open,2023-03-30,Masters 1000,Outdoor,Hard,Quarterfinals,3,Cerundolo F.,Khachanov K.,Khachanov K.,31,16,1320,2505,3.20,1.36,3-6 2-6
25358,Miami Open,2023-03-31,Masters 1000,Outdoor,Hard,Quarterfinals,3,Alcaraz C.,Fritz T.,Alcaraz C.,1,10,7420,2975,1.22,4.33,6-4 6-2
25359,Miami Open,2023-03-31,Masters 1000,Outdoor,Hard,Semifinals,3,Khachanov K.,Medvedev D.,Medvedev D.,16,5,2505,4330,4.50,1.20,6-7 6-3 3-6
25360,Miami Open,2023-04-01,Masters 1000,Outdoor,Hard,Semifinals,3,Sinner J.,Alcaraz C.,Sinner J.,11,1,2925,7420,3.00,1.40,6-7 6-4 6-2


In [108]:
#Filtering only for Hard surface data
hard = atp[atp['Surface'] == 'Hard']
hard

,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,score
0,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Mayer F.,Giraldo S.,Mayer F.,28,57,1215,778,1.36,3.00,6-4 6-4
1,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Benneteau J.,Nieminen J.,Nieminen J.,35,41,1075,927,2.20,1.61,3-6 6-2 1-6
2,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Nishikori K.,Matosevic M.,Nishikori K.,19,49,1830,845,1.25,3.75,7-5 6-2
3,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Mitchell B.,Baghdatis M.,Baghdatis M.,326,36,137,1070,9.00,1.07,4-6 4-6
4,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3,Istomin D.,Klizan M.,Istomin D.,43,30,897,1175,1.90,1.80,6-1 6-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25357,Miami Open,2023-03-30,Masters 1000,Outdoor,Hard,Quarterfinals,3,Cerundolo F.,Khachanov K.,Khachanov K.,31,16,1320,2505,3.20,1.36,3-6 2-6
25358,Miami Open,2023-03-31,Masters 1000,Outdoor,Hard,Quarterfinals,3,Alcaraz C.,Fritz T.,Alcaraz C.,1,10,7420,2975,1.22,4.33,6-4 6-2
25359,Miami Open,2023-03-31,Masters 1000,Outdoor,Hard,Semifinals,3,Khachanov K.,Medvedev D.,Medvedev D.,16,5,2505,4330,4.50,1.20,6-7 6-3 3-6
25360,Miami Open,2023-04-01,Masters 1000,Outdoor,Hard,Semifinals,3,Sinner J.,Alcaraz C.,Sinner J.,11,1,2925,7420,3.00,1.40,6-7 6-4 6-2


What is the lowest ranked player competing in each round of the ATP Tour Hard Surface?

In [109]:
r_atp = hard.groupby('Round')
# Use a dictionary to specify aggregation functions for each column
agg_dict = {
    'Rank_1': ['max', 'min'],
    'Rank_2': ['max', 'min']
}

# Apply .agg() using the dictionary
atp_agg = r_atp.agg(agg_dict)
#atp_agg.rename(columns={'Rank_1': 'Rank 1'}, inplace=True)
#rename for all other columns

# Display the aggregated DataFrame
atp_agg

Rank_1     Rank_2    
                 max min    max min
Round                              
1st Round       2146   1   1690   1
2nd Round       1761   1   1045   1
3rd Round        832   1    256   1
4th Round        144   1    192   1
Quarterfinals    837   1   1042   1
Round Robin       11   1     12   1
Semifinals      1042   1    454   1
The Final        454   1    167   1

In [115]:
#Who is this 454 ranked player?
lucky_player = hard[(hard['Round'] == 'The Final') & (hard['Rank_1'] == 454)]
lucky_player

,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,score
12297,Abierto Mexicano Mifel,2017-08-06,ATP250,Outdoor,Hard,The Final,3,Kokkinakis T.,Querrey S.,Querrey S.,454,24,85,1740,2.75,1.44,3-6 6-3 2-6


In [130]:
def percen(x):
    return x.count()/atp['Surface'].count()*100
#What percentage of matches are hard, grass and clay from ATP Tour 2013-2023?
atp_example = atp.groupby('Surface').agg({'Rank_1':percen})

atp_example

,Rank_1
Surface,
Clay,30.415582
Grass,11.103225
Hard,58.481192


## Reshaping

### Pandas Pivot and Pivot Table
Pandas provides pivot() and pivot_table() functions to reshape data into a summarized table for analysis. Let's explore how to use these functions with some examples.

#### Pivot
The pivot() function is used to reshape a DataFrame by converting column values into index values.

**Syntax:**

`df.pivot(index=None, columns=None, values=None)`

**Parameters:**

* index - Column to use as the row index.
* columns - Column to use as the column index.
* values - Column to aggregate.

**Returns:**

Pivoted dataframe.

In [131]:
import pandas as pd

data = {'Brand': ['Toyota','Honda','Toyota','Ford','Honda','Toyota'], 
        'Model': ['Corolla','Civic','Camry','Focus','Accord','Prius'],
        'Year': [2018,2019,2020,2018,2019,2021],
        'Price': [20000,22000,25000,21000,24000,28000],
        'Kilometers':[30000,27000,20000,26000,25000,19000]}

df = pd.DataFrame(data)
df

,Brand,Model,Year,Price,Kilometers
0,Toyota,Corolla,2018,20000,30000
1,Honda,Civic,2019,22000,27000
2,Toyota,Camry,2020,25000,20000
3,Ford,Focus,2018,21000,26000
4,Honda,Accord,2019,24000,25000
5,Toyota,Prius,2021,28000,19000


Now we can pivot the DataFrame with 'Brand' as the index, 'Model' as the columns, and 'Price' as the values:

In [133]:
df1 = df.pivot(index='Brand', columns='Model', values='Price')
df1.fillna(0, inplace=True)
df1

Model,Accord,Camry,Civic,Corolla,Focus,Prius
Brand,,,,,,
Ford,0.0,0.0,0.0,0.0,21000.0,0.0
Honda,24000.0,0.0,22000.0,0.0,0.0,0.0
Toyota,0.0,25000.0,0.0,20000.0,0.0,28000.0


#### Pivot Table
The pivot_table() function is similar to pivot() but provides more flexibility in calculating and aggregating data in the reshaped table.

**Syntax:**

`pivot_table(values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All')`

**Parameters:**


* values - Column to aggregate.
* index - Column(s) to be index.
* columns - Column(s) to be columns.
* aggfunc - Aggregation function like 'mean', 'sum', 'count'. Default is 'mean'.
* fill_value - Value to replace missing values with.
* margins - Add all row/column aggregates if True.
* dropna - Drop missing values. Default is True.
* margins_name - Name prefix for margin columns.

**Returns:**

Pivoted dataframe.

In [134]:
data = {'Brand': ['Toyota','Honda','Toyota','Ford','Honda','Toyota', 'Tesla', 'Toyota', 'Honda'], 
        'Model': ['Corolla','Civic','Camry','Focus','Accord','Prius', 'Model 3', 'RAV4', 'CR-V'],
        'Year': [2018,2019,2020,2018,2019,2021, 2020, 2019, 2020],
        'Price': [20000,22000,25000,21000,24000,28000, 50000, 28000, 27000],
        'Kilometers':[30000,27000,20000,26000,25000,19000, 10000, 31000, 22000]}

df = pd.DataFrame(data)
df

,Brand,Model,Year,Price,Kilometers
0,Toyota,Corolla,2018,20000,30000
1,Honda,Civic,2019,22000,27000
2,Toyota,Camry,2020,25000,20000
3,Ford,Focus,2018,21000,26000
4,Honda,Accord,2019,24000,25000
5,Toyota,Prius,2021,28000,19000
6,Tesla,Model 3,2020,50000,10000
7,Toyota,RAV4,2019,28000,31000
8,Honda,CR-V,2020,27000,22000


In [135]:
df.pivot_table(values='Price', index='Brand', columns='Model', aggfunc='mean')

Model,Accord,CR-V,Camry,Civic,Corolla,Focus,Model 3,Prius,RAV4
Brand,,,,,,,,,
Ford,NaN,NaN,NaN,NaN,NaN,21000.0,NaN,NaN,NaN
Honda,24000.0,27000.0,NaN,22000.0,NaN,NaN,NaN,NaN,NaN
Tesla,NaN,NaN,NaN,NaN,NaN,NaN,50000.0,NaN,NaN
Toyota,NaN,NaN,25000.0,NaN,20000.0,NaN,NaN,28000.0,28000.0


In [136]:
#Multiple aggregated values
df.pivot_table(values=['Price', 'Kilometers'], index='Brand', columns='Model',aggfunc=['mean', 'max'])

mean                                                        \
       Kilometers                                                         
Model      Accord     CR-V    Camry    Civic  Corolla    Focus  Model 3   
Brand                                                                     
Ford          NaN      NaN      NaN      NaN      NaN  26000.0      NaN   
Honda     25000.0  22000.0      NaN  27000.0      NaN      NaN      NaN   
Tesla         NaN      NaN      NaN      NaN      NaN      NaN  10000.0   
Toyota        NaN      NaN  20000.0      NaN  30000.0      NaN      NaN   

                                   ...        max                             \
                            Price  ... Kilometers    Price                     
Model     Prius     RAV4   Accord  ...       RAV4   Accord     CR-V    Camry   
Brand                              ...                                         
Ford        NaN      NaN      NaN  ...        NaN      NaN      NaN      NaN   
Honda       NaN      NaN  24000.0  ...        NaN  24000.0  27000.0      NaN   
Tesla       NaN      NaN      NaN  ...        NaN      NaN      NaN      NaN   
Toyota  19000.0  31000.0      NaN  ...    31000.0      NaN      NaN  25000.0   

                                                              
                                                              
Model     Civic  Corolla    Focus  Model 3    Prius     RAV4  
Brand                                                         
Ford        NaN      NaN  21000.0      NaN      NaN      NaN  
Honda   22000.0      NaN      NaN      NaN      NaN      NaN  
Tesla       NaN      NaN      NaN  50000.0      NaN      NaN  
Toyota      NaN  20000.0      NaN      NaN  28000.0  28000.0  

[4 rows x 36 columns]

In [137]:
# Adding new columns
df.pivot_table(values='Price', index='Brand', columns='Model', aggfunc='mean', margins=True)

Model,Accord,CR-V,Camry,Civic,Corolla,Focus,Model 3,Prius,RAV4,All
Brand,,,,,,,,,,
Ford,NaN,NaN,NaN,NaN,NaN,21000.0,NaN,NaN,NaN,21000.000000
Honda,24000.0,27000.0,NaN,22000.0,NaN,NaN,NaN,NaN,NaN,24333.333333
Tesla,NaN,NaN,NaN,NaN,NaN,NaN,50000.0,NaN,NaN,50000.000000
Toyota,NaN,NaN,25000.0,NaN,20000.0,NaN,NaN,28000.0,28000.0,25250.000000
All,24000.0,27000.0,25000.0,22000.0,20000.0,21000.0,50000.0,28000.0,28000.0,27222.222222


### Webscrapping OCU's sports statistics

In [ ]:
# URL of the webpage containing the HTML table(s)
url = 'https://gomightyoaks.com/sports/baseball/stats/2022'

# Read the HTML tables from the webpage
baseball = pd.read_html(url)

# Depending on the webpage structure, there might be multiple tables
# You can access each table using tables[index], where index is the index of the table you want to extract

# For example, to extract the first table
ocu = baseball[0]
ocu

In [ ]:
ocu.pivot_table(values='R', index='Opponent', columns='W/L', aggfunc='mean', margins=True)

### Melt
The melt() function in pandas is used to unpivot or reshape your data from wide to long format. This allows you to reduce the number of columns in your dataframe by "melting" multiple columns into a single column.

**Syntax**

`df.melt(id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None)`

**Parameters:**

* id_vars : Column(s) to use as identifier variables.
* value_vars : Column(s) to unpivot. If not specified, uses all columns that are not set as id_vars.
* var_name : Name to use for the 'variable' column. Default is 'variable'.
* value_name : Name to use for the 'value' column. Default is 'value'.
* col_level : If columns are a MultiIndex then use this level to melt.

**Returns:**

Pivoted DataFrame in long format.

In [ ]:
cars = pd.DataFrame({'Car Model': ['Prius', 'CX-5', 'Tesla Model 3', 'Camry'],
                  'MPG': [50, 25, 30, 28],
                  'Horsepower': [120, 187, 258, 203], 
                  'Weight': [3000, 3500, 4079, 3300]})
cars

In [ ]:
melted_df = cars.melt(id_vars='Car Model', 
                    value_vars=['MPG', 'Horsepower', 'Weight'])

melted_df

In [ ]:
#Renaming columns
melted_df = cars.melt(id_vars='Car Model', 
                    value_vars=['MPG', 'Horsepower', 'Weight'],
                    var_name='Measurement',
                    value_name='Value')

melted_df